In [50]:
import pandas as pd
import fasttext
import numpy as np
from nltk.corpus import stopwords

In [51]:
!pwd

/Users/pree_samurai/Desktop/Hate-Speech-Project/jigsaw-toxic-comment-classification-challenge


In [92]:
train_data_fr = pd.read_csv("train.csv")
train_data_fr

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [93]:
train_data_fr["verdict"] = train_data_fr[list(train_data_fr.columns)[3:]].sum(axis=1)
train_data_fr

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,verdict
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,0


In [94]:
train_data_fr = train_data_fr.drop(["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"], axis=1)
train_data_fr

,comment_text,verdict
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [95]:
attack = pd.read_csv("../../Cloud_Project/attack_parsed_dataset.csv")
aggression = pd.read_csv("../../Cloud_Project/aggression_parsed_dataset.csv")
kaggle = pd.read_csv("../../Cloud_Project/kaggle_parsed_dataset.csv")
toxic = pd.read_csv("../../Cloud_Project/toxicity_parsed_dataset.csv")
# twitter_racist = pd.read_csv("../../Cloud_Project/twitter_racism_parsed_dataset.csv")
# twitter_sexist = pd.read_csv("../../Cloud_Project/twitter_sexism_parsed_dataset.csv")
twitter_p = pd.read_csv("../../Cloud_Project/twitter_parsed_dataset.csv")
youtube = pd.read_csv("../../Cloud_Project/youtube_parsed_dataset.csv")
reddit = pd.read_csv("../../Cloud_Project/reddit.csv")
og_co = pd.read_csv("../../Cloud_Project/e_train.csv")
hate = pd.read_csv("../../Cloud_Project/bro_please_work.csv")

In [96]:
def clean(stuff):
    for each in stuff:
        each["comment_text"] = each["Text"]
        each["verdict"] = each["oh_label"]
        each = each[["comment_text", "verdict"]]
    print("BOXED")

reddit["Text"] = reddit["tweet"]
reddit = reddit.drop(["tweet"], axis=1)
reddit["oh_label"] = reddit["class"].replace(to_replace = 0, value = 1, inplace=True)
reddit["oh_label"] = reddit["class"].replace(to_replace = 2, value = 0, inplace=True)
reddit["oh_label"] = reddit["class"].astype(float)
reddit = reddit.drop(["class"], axis=1)
reddit

,Unnamed: 0,count,hate_speech,offensive_language,neither,Text,oh_label
0,0,3,0,0,3,!!! RT @mayasolovely: As a woman you shouldn't...,0.0
1,1,3,0,3,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1.0
2,2,3,0,3,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1.0
3,3,3,0,2,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1.0
4,4,6,0,6,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1.0
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1.0
24779,25292,3,0,1,2,"you've gone and broke the wrong heart baby, an...",0.0
24780,25294,3,0,3,0,young buck wanna eat!!.. dat nigguh like I ain...,1.0
24781,25295,6,0,6,0,youu got wild bitches tellin you lies,1.0


In [118]:
shit = [attack, aggression, kaggle, toxic, twitter_p, youtube, reddit, hate] 
clean(shit)
train_data = pd.concat([train_data_fr, attack, aggression, og_co, kaggle, toxic, twitter_p, youtube, reddit])

BOXED


In [119]:
train_data = train_data[["comment_text", "verdict"]]
# train_data["label"] = "__label__"+str(train_data["verdict"])
# train_data["label"] = train_data["label"][:10]
# train_data
train_data

,comment_text,verdict
0,Explanation\nWhy the edits made under my usern...,0.0
1,D'aww! He matches this background colour I'm s...,0.0
2,"Hey man, I'm really not trying to edit war. It...",0.0
3,"""\nMore\nI can't make any real suggestions on ...",0.0
4,"You, sir, are my hero. Any chance you remember...",0.0
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1.0
24779,"you've gone and broke the wrong heart baby, an...",0.0
24780,young buck wanna eat!!.. dat nigguh like I ain...,1.0
24781,youu got wild bitches tellin you lies,1.0


In [120]:
labels = train_data["verdict"].tolist()
labels = [0 if each==0 else 1 for each in labels]
labels = ["__label__"+str(each) for each in labels]
train_data["label"] = labels
train_data["text"] = train_data["comment_text"]
train_data = train_data.drop(["comment_text", "verdict"], axis=1)
tmp = train_data["text"].tolist()
tmp = [str(each).lower() for each in tmp]
train_data["text"] = tmp
train_data

,label,text
0,__label__0,explanation\nwhy the edits made under my usern...
1,__label__0,d'aww! he matches this background colour i'm s...
2,__label__0,"hey man, i'm really not trying to edit war. it..."
3,__label__0,"""\nmore\ni can't make any real suggestions on ..."
4,__label__0,"you, sir, are my hero. any chance you remember..."
...,...,...
24778,__label__1,you's a muthaf***in lie &#8220;@lifeasking: @2...
24779,__label__0,"you've gone and broke the wrong heart baby, an..."
24780,__label__1,young buck wanna eat!!.. dat nigguh like i ain...
24781,__label__1,youu got wild bitches tellin you lies


In [207]:
puncts = set([',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '==', '¦', '║', '―', '¥', '▓', '—', '‹', '─',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ])

def clean_text(data):
    stop = stopwords.words('english')
    res = []
    data['text'] = data['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))
    for x in data['text']:
        #print(x)
        for punct in puncts:
            if punct in x:
                    x = x.replace(punct,' ')
        res.append(x)
    return res

In [208]:
train_data["text"] = clean_text(train_data)

In [209]:
np.savetxt(r'train.txt', train_data.values, fmt='%s')

In [210]:
model = fasttext.train_supervised('train.txt', ws=1, wordNgrams=2, minn=2, maxn=20, lr=0.4, epoch=25, loss='hs')
model.save_model('trained.ftz')

Read 22M words
Number of words:  278585
Number of labels: 2
Progress: 100.0% words/sec/thread:  229415 lr:  0.000000 avg.loss:  0.107074 ETA:   0h 0m 0s  2.7% words/sec/thread:  242428 lr:  0.389364 avg.loss:  0.119761 ETA:   0h 4m 7s


In [242]:
res = model.predict('whore')
"hate" if res[0][0][-1]=='1' else "no hate"

'hate'

In [118]:
test_data = pd.read_csv("test.csv")
test_labels = pd.read_csv("test_labels.csv")
test_data = pd.merge(test_data, test_labels, how="inner")
test_data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",-1,-1,-1,-1,-1,-1


In [119]:
test_data["verdict"] = test_data[list(test_data.columns)[3:]].sum(axis=1)
test_data = test_data.drop(["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"], axis=1)
labels = test_data["verdict"].tolist()
labels = [0 if each==0 else 1 for each in labels]
labels = ["__label__"+str(each) for each in labels]
test_data["label"] = labels
test_data["text"] = test_data["comment_text"]
test_data = test_data.drop(["comment_text", "verdict"], axis=1)
np.savetxt(r'test.txt', test_data.values, fmt='%s')
test_data

,label,text
0,__label__1,Yo bitch Ja Rule is more succesful then you'll...
1,__label__1,== From RfC == \n\n The title is fine as it is...
2,__label__1,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,__label__1,":If you have a look back at the source, the in..."
4,__label__1,I don't anonymously edit articles at all.
...,...,...
153159,__label__1,". \n i totally agree, this stuff is nothing bu..."
153160,__label__1,== Throw from out field to home plate. == \n\n...
153161,__label__1,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,__label__1,""" \n\n == """"One of the founding nations of the..."


In [120]:
model.test_label("test.txt")

{'__label__1': {'precision': 0.9248826291079812,
  'recall': nan,
  'f1score': 1.8497652582159625},
 '__label__0': {'precision': 0.41050090947250595,
  'recall': nan,
  'f1score': 0.8210018189450119}}